# NOTEBOOK PARA REALIZAR TODO EL ANÁLISIS DE SECUENCIACIÓN, ENSAMBLE Y ANÁLISIS DE GENOMAS DE VIRUS DE INFLUENZA



## LIMPIEZA DE ARCHIVOS FASTQ

Definir variables útiles

In [ ]:
MUESTRA="CPA_TEST"

Cambiar el directorio a la carpeta a procesar. Ya debe contener los archivos FASTQ a procesar.

In [ ]:
cd /home/admcenapa/DAVID_RENDON/TRABAJO/2025/${MUESTRA}

Crear una carpeta para depositar las secuencias crudas. Ahí se moverán los archivos crudos.

In [ ]:
mkdir RAW
mv CPA* ./RAW

Crear ligas simbolicas a esos archivos en la carpeta actual para facilitar operaciones futuras.

In [ ]:
ln -s ./RAW/CPA-00245-25-P1-1_S2_L001_R1_001.fastq.gz R1.fq.gz
ln -s ./RAW/CPA-00245-25-P1-1_S2_L001_R2_001.fastq.gz R2.fq.gz

Crear un directorio para depositar el análisis de calidad de lecturas crudas:

In [ ]:
mkdir CALIDAD_CRUDA

Ejecutar el análisis de calidad. Usar ambiente calidad

In [ ]:
conda activate calidad

In [ ]:
fastqc R1.fq.gz R2.fq.gz --outdir ./CALIDAD_CRUDA --threads 14 

Revisar los archivos HTML de salida para identificar parámetros que ayuden a la limpieza. Esto se puede lograr transfiriendo los archivos HTML a una computadora local y visualizando en un navedador de internet.

Ejecutar el programa de limpieza. Ajustar parámetros de acuerdo a las necesidades. Cuidado al modificar los cores (ver documentación). Los resultados se guardarán en ./TRIMMING/


In [ ]:
trim_galore --paired --retain_unpaired --gzip --fastqc --cores 6 --clip_R1 16 --clip_R2 16 --three_prime_clip_R1 2 --three_prime_clip_R2 2 --length 100 -o TRIMMING/ R1.fq.gz R2.fq.gz

In [ ]:
conda deactivate

Revisar los archivos HTML de salida para verificar los resultados de la limpieza. Queda a consideración de la experiencia del usuario el quedar conforme con la limpieza realizada.

## ALINEAMIENTO DE LECTURAS CONTRA BASE DE DATOS DE INFLUENZA

Hacer directorio para los alineamientos:

In [ ]:
mkdir -p ALINEAMIENTO/BWA/ALL_SEGMENTS_MAPPING

Crear ligas simbolicas a los archivos a usar:

In [ ]:
ln -s TRIMMING/R1_val_1.fq.gz reads_r1_tr
ln -s TRIMMING/R2_val_2.fq.gz reads_r2_tr
ln -s TRIMMING/R1_unpaired_1.fq.gz reads_u1_tr
ln -s TRIMMING/R2_unpaired_2.fq.gz reads_u2_tr

Activar ambiente conda alineamiento

In [ ]:
conda activate alineamiento

Hacer los alineamientos de las secuencias pareadas y no pareadas contra la base de datos viral preparada en la UASIP.

In [ ]:
#Alineamiento de secuencias pareadas 
bwa-mem2 mem -t 20 /backup/DATABASES/UASIP/BWAMEM2/INFLUENZA/influenza_db.fna reads_r1_tr reads_r2_tr >ALINEAMIENTO/BWA/ALL_SEGMENTS_MAPPING/paired.sam 

#Alineamiento de secuencias no pareadas de R1
bwa-mem2 mem -t 20 /backup/DATABASES/UASIP/BWAMEM2/INFLUENZA/influenza_db.fna reads_u1_tr >ALINEAMIENTO/BWA/ALL_SEGMENTS_MAPPING/unpaired_r1.sam

#Alineamiento de secuencias no pareadas de R2
bwa-mem2 mem -t 20 /backup/DATABASES/UASIP/BWAMEM2/INFLUENZA/influenza_db.fna reads_u2_tr >ALINEAMIENTO/BWA/ALL_SEGMENTS_MAPPING/unpaired_r2.sam

Entrar al directorio donde se depositaron los resultados del alineamiento. Formatear los archivos sam resultantes en un archivo bam ordenado. Regresar al directorio del proyecto.

In [ ]:
cd ALINEAMIENTO/BWA/ALL_SEGMENTS_MAPPING 
samtools merge -u - paired.sam unpaired_r1.sam unpaired_r2.sam | samtools sort -o all_sorted.bam && rm paired.sam unpaired_r1.sam unpaired_r2.sam
cd -

Crear ocho directorios para depositar las lecturas por el segmento al cual alinearon.

In [ ]:
mkdir ALINEAMIENTO/BWA/S{1..8}

Ejecutar el siguiente programa. Este programa abre el archivo bam, elimina alineamientos con alineamientos secundarios, y obtiene las lecturas por cada segmento. Se analiza, por segmento, el formato CIGAR asociado de cada lectura. Se filtran aquellas secuencias que alinearon en al menos 70% de sus bases. Se guardan los nombres de las lecturas pertinentes y se extraen las secuencias de los archivos fastq limpios usando dichos nombres. Los archivos producidos se guardan en las ubicaciones pertinentes.

In [ ]:
~/analisis_influenza/bin/separar_seqs_segmentos.sh

## ENSAMBLE DE NOVO DE GENOMAS DE INFLUENZA

Activar ambiente conda de los ensambladores

In [ ]:
conda activate ensamble

Activa una terminal tmux para que se encargue de enviar los procesos aunque salgamos del servidor

In [ ]:
tmux new -s ensamblaje

Pegar el siguiente código de instrucciones en la terminal tmux. Este paso está diseñado para ejecutarse sobre todos los segmentos virales. Es importante revisar cada archivo log de cada subcarpeta para verificar que se haya ejecutado todos los procesos, desde el kmer 11 al kmer 127. Se hace en terminal tmux para que el ciclo no caiga cuando se termine la conexion con el servidor.

In [ ]:
for i in {1..8}; do
    ID="S${i}"
    id="s${i}"
    LOG="./ENSAMBLE/SPADES/${ID}/log_11_127_${ID}"
    mkdir -p "./ENSAMBLE/SPADES/${ID}"

    TMP_SCRIPT=$(mktemp /tmp/run_${ID}_XXXXXX.sh)

    cat > "$TMP_SCRIPT" <<EOF
#!/bin/bash

{
    echo "===== Iniciando proceso para ${ID} ====="
    echo "Inicio: \$(date)"
} >> "$LOG"

~/analisis_influenza/bin/ensamble.sh \\
    -a "./ALINEAMIENTO/BWA/${ID}/${id}_reads_r1.fq.gz" \\
    -b "./ALINEAMIENTO/BWA/${ID}/${id}_reads_r2.fq.gz" \\
    -x "./ALINEAMIENTO/BWA/${ID}/${id}_reads_u1.fq.gz" \\
    -y "./ALINEAMIENTO/BWA/${ID}/${id}_reads_u2.fq.gz" \\
    -t 26 --kini 11 --kfin 127 -o ./ENSAMBLE/SPADES/${ID}/OUT_11_127 >> "$LOG" 2>&1

{
    echo "Fin: \$(date)"
    echo "===== Finalizó proceso para ${ID} ====="
    echo
} >> "$LOG"

rm -- "\$0"
EOF

    chmod +x "$TMP_SCRIPT"
    nohup "$TMP_SCRIPT" &
    wait
done

In [ ]:
cd /home/admcenapa/DAVID_RENDON/TRABAJO/2025/${MUESTRA}

Se puede cerrar sesion con Ctrl + B, luego suelta y presiona D. Para reconectar la terminal tmux usar: tmux attach -t ensamblaje

Monitorear que el proceso haya concluido, revisando periodicamente los archivos log. Una vez concluido el proceso de tmux, entrar a la terminal tmux y cerrar con: exit

Activar ambiente alineamiento para continuar con el procesamiento

In [10]:
conda activate alineamiento

Hacer loop para unir todos los resultados de blast (si se hizo por partes) y analizar los resultados, extraer secuencias, revisar orientacion y calcular profundidad.

In [11]:
for i in {1..8}; do
    ID="S${i}"

    cd "./ENSAMBLE/SPADES/${ID}"

    ~/analisis_influenza/bin/merge_blast_results.sh

    ~/analisis_influenza/bin/ensamble_analyze_blast.sh ${ID}_blastn-careful_merged.txt
    
    cd -
done

🔍 Procesando S1 en /home/admcenapa/DAVID_RENDON/TRABAJO/2025/CPA_TEST/ENSAMBLE/SPADES/S1
	📄 Añadiendo: OUT_11_127/BLAST_RESULTS/blastn-careful.txt
	✅ Archivo combinado creado: S1_blastn-careful_merged.txt
✅ Resultados guardados en: best_result_S1_blastn-careful_merged.txt
✅ Secuencia extraída en: best_result_S1_sequence.fna
✅ Manteniendo la secuencia original (orientación '+')...
✅ Secuencia corregida guardada en: best_result_S1_sequence_corrected.fna
➤ Calculando profundidad de lecturas de S1 en mejor secuencia ensamblada.
	➤ Preparando indice
	➤ Alineando lecturas pareadas de S1 en mejor secuencia ensamblada.
	➤ Alineando lecturas no pareadas de R1 de S1 en mejor secuencia ensamblada.
	➤ Alineando lecturas no pareadas de R2 de S1 en mejor secuencia ensamblada.
[W::bam_merge_core2] No @HD tag found.
	➤ Calculando profundidades
/home/admcenapa/DAVID_RENDON/TRABAJO/2025/CPA_TEST
🔍 Procesando S2 en /home/admcenapa/DAVID_RENDON/TRABAJO/2025/CPA_TEST/ENSAMBLE/SPADES/S2
	📄 Añadiendo: OUT_11

Ingresar al directorio SPADES para unir profundidades y calcular lecturas utilizadas.

In [12]:
cd ./ENSAMBLE/SPADES

In [13]:
~/analisis_influenza/bin/unir_profundidades.sh \
    --S1 S1/PROFUNDIDAD/S1_profundidad \
    --S2 S2/PROFUNDIDAD/S2_profundidad \
    --S3 S3/PROFUNDIDAD/S3_profundidad \
    --S4 S4/PROFUNDIDAD/S4_profundidad \
    --S5 S5/PROFUNDIDAD/S5_profundidad \
    --S6 S6/PROFUNDIDAD/S6_profundidad \
    --S7 S7/PROFUNDIDAD/S7_profundidad \
    --S8 S8/PROFUNDIDAD/S8_profundidad

~/analisis_influenza/bin/ensamble_cal_porc_lect_mapeadas.sh

📋 Calculando porcentaje de lecturas usadas en ensamblaje de novo para cada segmento (S1-S8)...
Segmento  	Lecturas_alineadas  	Lecturas_ensamblaje 	Porcentaje_usadas   
S1        	25237               	23634               	93.65               
S2        	24892               	20643               	82.93               
S3        	22049               	21458               	97.32               
S4        	81650               	79377               	97.22               
S5        	95246               	90403               	94.92               
S6        	101788              	95902               	94.22               
S7        	185482              	181380              	97.79               
S8        	175278              	171412              	97.79               
✅ Resultados guardados en 'uso_de_lecturas.tsv'.


Activar ambiente R

In [14]:
conda activate R

Graficar profundidades.

In [17]:
~/analisis_influenza/bin/graficar_profundidad.R --input_file profundidades_finales.tsv --muestra CPA_TEST

✅ Archivo leído correctamente: profundidades_finales.tsv
- Número de filas: 2347
- Número de columnas: 9
- Muestra procesada: CPA_TEST
💻 Limpiando y procesando
📄 Gráfica producida. Finalizando



Graficar uso de secuencias por segmento.

In [18]:
~/analisis_influenza/bin/graficar_lecturas.R --input_file uso_de_lecturas.tsv --muestra CPA_TEST

✅ Archivo leído correctamente: uso_de_lecturas.tsv
- Muestra procesada: CPA_TEST
💻 Limpiando y procesando
📄 Gráfica producida. Finalizando



In [19]:
conda deactivate

In [16]:
pwd

/home/admcenapa/DAVID_RENDON/TRABAJO/2025/CPA_TEST/ENSAMBLE/SPADES
